## ETL
#### El dataset provisto por el Observatorio de movilidad y seguridad vial, (OMSV), contiene información sobre accidentes de tránsito. Específicamente se compone de dos hojas dentro de un formato de libro Excel, una sobre los hechos del accidente y otra con datos sobre las víctimas de tales accidentes.
#### Tras haber explorado la estructura de los y características generales en el notebook "1 Análisis inicial", debemos limpiar, organizar y cargar los datos para hacer el análisis multivariado pertinente en el EDA, BI y cumplir los KPIs propuestos.

In [1]:
# Importamos las librerías necesarias para trabajar con nuestro archivo
import numpy as np
import pandas as pd
import warnings
import datetime
from bs4 import BeautifulSoup 
import requests
warnings.filterwarnings("ignore")

In [2]:
# Leemos el archivo y sus hojas
df_excel_homicidios = pd.ExcelFile("Data/homicidios.xlsx")
df_hechos = pd.read_excel(df_excel_homicidios, "HECHOS")
df_victimas = pd.read_excel(df_excel_homicidios, "VICTIMAS")

In [3]:
# Visualizamos la hoja "HECHOS" del libro "homicidios"
df_hechos.head(2)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS


In [4]:
# Visualizamos información sobre la hoja
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

#### Algunas de las columnas de la hoja "hechos" contienen información que se encuentra agrupada en otras columnas, por tanto, decidimos eliminar información redundante.

In [5]:
df_hechos = df_hechos.drop(columns=["AAAA", "MM", "DD", "HH", "Dirección Normalizada", "Calle", "Altura", "Cruce", "XY (CABA)"])
df_hechos.head(2)

,ID,N_VICTIMAS,FECHA,HORA,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,04:00:00,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,8,-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,01:15:00,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,9,-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS


In [6]:
# Visualizamos información sobre la hoja
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ID               696 non-null    object        
 1   N_VICTIMAS       696 non-null    int64         
 2   FECHA            696 non-null    datetime64[ns]
 3   HORA             696 non-null    object        
 4   LUGAR_DEL_HECHO  696 non-null    object        
 5   TIPO_DE_CALLE    696 non-null    object        
 6   COMUNA           696 non-null    int64         
 7   pos x            696 non-null    object        
 8   pos y            696 non-null    object        
 9   PARTICIPANTES    696 non-null    object        
 10  VICTIMA          696 non-null    object        
 11  ACUSADO          696 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 65.4+ KB


#### Nos disponemos a analizar la información de la hoja "victimas" para ver si resulta conveniente agrupar las dos hojas en un solo dataframe que disponibilizaremos para el EDA.

In [7]:
# Visualizamos la hoja "VICTIMAS" del libro "homicidios"
df_victimas.head(2)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00


#### Esta hoja presenta la misma característica que la anterior, tener columnas agrupadas en otras, por lo que el procedimiento puede ser el mismo.

In [8]:
df_victimas = df_victimas.drop(columns=["MM", "DD", "FECHA"])
df_victimas.head(2)

,ID_hecho,AAAA,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00


#### Renombramos la columna "AAAA" a "AÑO" y la convertimos a tipo de dato int.

In [9]:
# Renombramos la columna 'AAAA' a 'AÑO'
df_victimas = df_victimas.rename(columns={"AAAA": "AÑO"})

# Convertir la columna 'AÑO' a tipo de dato int
df_victimas["AÑO"] = df_victimas["AÑO"].astype(int)

#### Las columna "ID" de la hoja "hechos" coincide con la columna "ID_hecho" de la hoja "victimas". Por ende, debemos hacer el merge a partir de dichas columnas.

In [10]:
df_accidentes = pd.merge(df_hechos, df_victimas, left_on="ID", right_on="ID_hecho")
df_accidentes.head(2)

,ID,N_VICTIMAS,FECHA,HORA,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA_x,ACUSADO,ID_hecho,AÑO,ROL,VICTIMA_y,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1,2016-01-01,04:00:00,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,8,-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO,2016-0001,2016,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,1,2016-01-02,01:15:00,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,9,-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS,2016-0002,2016,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00


#### En este punto notamos que los valores nulos están registrados como "SD", equivalente a "Sin Datos". Los reemplazaremos por valores nulos para no sesgar la visualización.

In [11]:
def reemplazar_SD(df):
    for i in range(len(df)):
        for col in df.columns:
            if df.loc[i, col] == "SD":
                df.loc[i, col] = pd.NaT
    return df

reemplazar_SD(df_accidentes)

,ID,N_VICTIMAS,FECHA,HORA,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA_x,ACUSADO,ID_hecho,AÑO,ROL,VICTIMA_y,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1,2016-01-01,04:00:00,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,8,-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO,2016-0001,2016,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,1,2016-01-02,01:15:00,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,9,-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS,2016-0002,2016,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,1,2016-01-03,07:00:00,AV ENTRE RIOS 2034,AVENIDA,1,-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO,2016-0003,2016,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,1,2016-01-10,00:00:00,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,8,-58.46503904,-34.68092974,MOTO-SD,MOTO,NaT,2016-0004,2016,CONDUCTOR,MOTO,MASCULINO,18,NaT
4,2016-0005,1,2016-01-21,05:20:00,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,1,-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS,2016-0005,2016,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2021-0093,1,2021-12-13,17:10:00,AV. RIESTRA Y MOM,AVENIDA,7,-58.43353773,-34.64561636,MOTO-AUTO,MOTO,AUTO,2021-0093,2021,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18,2021-12-18 00:00:00
713,2021-0094,1,2021-12-20,01:10:00,AU DELLEPIANE Y LACARRA,AUTOPISTA,9,-58.46739825,-34.65117757,MOTO-AUTO,MOTO,AUTO,2021-0094,2021,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43,2021-12-20 00:00:00
714,2021-0095,1,2021-12-30,00:43:00,AV. GAONA Y TERRADA,AVENIDA,11,-58.47293407,-34.61984745,MOTO-CARGAS,MOTO,CARGAS,2021-0095,2021,CONDUCTOR,MOTO,MASCULINO,27,2022-01-02 00:00:00
715,2021-0096,1,2021-12-15,10:30:00,AV. EVA PERON 4071,AVENIDA,9,-58.47066794,-34.65021673,AUTO-CARGAS,AUTO,CARGAS,2021-0096,2021,CONDUCTOR,AUTO,MASCULINO,60,2021-12-20 00:00:00


In [12]:
df_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   717 non-null    object        
 1   N_VICTIMAS           717 non-null    int64         
 2   FECHA                717 non-null    datetime64[ns]
 3   HORA                 716 non-null    object        
 4   LUGAR_DEL_HECHO      716 non-null    object        
 5   TIPO_DE_CALLE        717 non-null    object        
 6   COMUNA               717 non-null    int64         
 7   pos x                717 non-null    object        
 8   pos y                717 non-null    object        
 9   PARTICIPANTES        717 non-null    object        
 10  VICTIMA_x            708 non-null    object        
 11  ACUSADO              694 non-null    object        
 12  ID_hecho             717 non-null    object        
 13  AÑO                  717 non-null  

#### Después de hacer el merged, vemos que hay columnas con información repetida, seguimos con la eliminación.

In [13]:
df_accidentes = df_accidentes.drop(columns=["ID_hecho", "VICTIMA_y"])
df_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   717 non-null    object        
 1   N_VICTIMAS           717 non-null    int64         
 2   FECHA                717 non-null    datetime64[ns]
 3   HORA                 716 non-null    object        
 4   LUGAR_DEL_HECHO      716 non-null    object        
 5   TIPO_DE_CALLE        717 non-null    object        
 6   COMUNA               717 non-null    int64         
 7   pos x                717 non-null    object        
 8   pos y                717 non-null    object        
 9   PARTICIPANTES        717 non-null    object        
 10  VICTIMA_x            708 non-null    object        
 11  ACUSADO              694 non-null    object        
 12  AÑO                  717 non-null    int32         
 13  ROL                  706 non-null  

#### En este punto podemos empezar a tratar valores faltantes en las columnas.

In [14]:
# Eliminamos un registro del que no se conoce la hora del suceso
df_accidentes = df_accidentes.dropna(subset=["HORA"])
df_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 716 entries, 0 to 716
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   716 non-null    object        
 1   N_VICTIMAS           716 non-null    int64         
 2   FECHA                716 non-null    datetime64[ns]
 3   HORA                 716 non-null    object        
 4   LUGAR_DEL_HECHO      715 non-null    object        
 5   TIPO_DE_CALLE        716 non-null    object        
 6   COMUNA               716 non-null    int64         
 7   pos x                716 non-null    object        
 8   pos y                716 non-null    object        
 9   PARTICIPANTES        716 non-null    object        
 10  VICTIMA_x            707 non-null    object        
 11  ACUSADO              693 non-null    object        
 12  AÑO                  716 non-null    int32         
 13  ROL                  705 non-null    obj

#### Eliminamos un registro que no tiene lugar del hecho.

In [15]:
df_accidentes = df_accidentes.dropna(subset=["LUGAR_DEL_HECHO"])
df_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 715 entries, 0 to 716
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   715 non-null    object        
 1   N_VICTIMAS           715 non-null    int64         
 2   FECHA                715 non-null    datetime64[ns]
 3   HORA                 715 non-null    object        
 4   LUGAR_DEL_HECHO      715 non-null    object        
 5   TIPO_DE_CALLE        715 non-null    object        
 6   COMUNA               715 non-null    int64         
 7   pos x                715 non-null    object        
 8   pos y                715 non-null    object        
 9   PARTICIPANTES        715 non-null    object        
 10  VICTIMA_x            706 non-null    object        
 11  ACUSADO              693 non-null    object        
 12  AÑO                  715 non-null    int32         
 13  ROL                  704 non-null    obj

#### En las columnas "VICTIMA_X", "ACUSADO" y "ROL" tenemos valores nulos a los que podemos imputarles el valor "No identificado", tratando de conervar la información del registro.

In [16]:
# Completamos los valores nulos de las columnas "VICTIMA", "ACUSADO" y "ROL" por "No identificado"
df_accidentes["VICTIMA_x"].fillna("No identificado", inplace=True)
df_accidentes["ACUSADO"].fillna("No identificado", inplace=True)
df_accidentes["ROL"].fillna("No identificado", inplace=True)
df_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 715 entries, 0 to 716
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   715 non-null    object        
 1   N_VICTIMAS           715 non-null    int64         
 2   FECHA                715 non-null    datetime64[ns]
 3   HORA                 715 non-null    object        
 4   LUGAR_DEL_HECHO      715 non-null    object        
 5   TIPO_DE_CALLE        715 non-null    object        
 6   COMUNA               715 non-null    int64         
 7   pos x                715 non-null    object        
 8   pos y                715 non-null    object        
 9   PARTICIPANTES        715 non-null    object        
 10  VICTIMA_x            715 non-null    object        
 11  ACUSADO              715 non-null    object        
 12  AÑO                  715 non-null    int32         
 13  ROL                  715 non-null    obj

#### A los valores nulos de la columna "SEXO" se les puede imputar el valor "No especificado", para conservar los registros para aquellas estadísticas que no requieran dichos valores.

In [17]:
# Rellenamos los valores nulos de la columna
df_accidentes["SEXO"].fillna("No especificado", inplace=True)
df_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 715 entries, 0 to 716
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   715 non-null    object        
 1   N_VICTIMAS           715 non-null    int64         
 2   FECHA                715 non-null    datetime64[ns]
 3   HORA                 715 non-null    object        
 4   LUGAR_DEL_HECHO      715 non-null    object        
 5   TIPO_DE_CALLE        715 non-null    object        
 6   COMUNA               715 non-null    int64         
 7   pos x                715 non-null    object        
 8   pos y                715 non-null    object        
 9   PARTICIPANTES        715 non-null    object        
 10  VICTIMA_x            715 non-null    object        
 11  ACUSADO              715 non-null    object        
 12  AÑO                  715 non-null    int32         
 13  ROL                  715 non-null    obj

#### Los valores nulos de la columna "EDAD" pueden suplirse con la media de todas las edades presentes en la columna. Además podemos agrupar los valores en rangos.

In [18]:
# Convertimos a tipo de dato numérico la columna, para después rellenar
df_accidentes["EDAD"] = pd.to_numeric(df_accidentes["EDAD"], errors="coerce")

# Calculamos la media
media_edad = df_accidentes["EDAD"].dropna().mean()

# Hacemos la imputación
df_accidentes["EDAD"].fillna(media_edad, inplace=True)

# Cambiamos el tipo de dato int
df_accidentes["EDAD"] = df_accidentes["EDAD"].astype(int)
df_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 715 entries, 0 to 716
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   715 non-null    object        
 1   N_VICTIMAS           715 non-null    int64         
 2   FECHA                715 non-null    datetime64[ns]
 3   HORA                 715 non-null    object        
 4   LUGAR_DEL_HECHO      715 non-null    object        
 5   TIPO_DE_CALLE        715 non-null    object        
 6   COMUNA               715 non-null    int64         
 7   pos x                715 non-null    object        
 8   pos y                715 non-null    object        
 9   PARTICIPANTES        715 non-null    object        
 10  VICTIMA_x            715 non-null    object        
 11  ACUSADO              715 non-null    object        
 12  AÑO                  715 non-null    int32         
 13  ROL                  715 non-null    obj

In [19]:
# Crear los límites de los rangos y las etiquetas correspondientes
bins = [0, 20, 40, 60, 80, float("inf")]
labels = ["menos de 20", "21-40", "41-60", "61-80", "más de 80"]

# Utilizar pd.cut para crear los rangos y asignar etiquetas
df_accidentes["EDAD"] = pd.cut(df_accidentes["EDAD"], bins=bins, labels=labels, right=False)
df_accidentes["EDAD"]

0      menos de 20
1            61-80
2            21-40
3      menos de 20
4            21-40
          ...     
712    menos de 20
713          41-60
714          21-40
715          61-80
716          41-60
Name: EDAD, Length: 715, dtype: category
Categories (5, object): ['menos de 20' < '21-40' < '41-60' < '61-80' < 'más de 80']

#### Los valores de la columna "FECHA_FALLECIMIENTO" que son nulos, se deben a que las víctimas fallecieron en el mismo instante que ocurrió el accidente, por tanto, podemos cambiar el formato de dicha columna y rellenar los valore faltantes con los de la columna "FECHA".

In [20]:
# Primero cambiamos los formatos a fecha estandar
df_accidentes["FECHA_FALLECIMIENTO"] = pd.to_datetime(df_accidentes["FECHA_FALLECIMIENTO"], format="%Y-%m-%d %H:%M:%S", errors="coerce")
df_accidentes["FECHA_FALLECIMIENTO"] = df_accidentes["FECHA_FALLECIMIENTO"].dt.strftime("%Y-%m-%d")

# Rellenamos los valores nulos de "FECHA_FALLECIMIENTO"
df_accidentes["FECHA_FALLECIMIENTO"].fillna(df_accidentes["FECHA"], inplace=True)
df_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 715 entries, 0 to 716
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   715 non-null    object        
 1   N_VICTIMAS           715 non-null    int64         
 2   FECHA                715 non-null    datetime64[ns]
 3   HORA                 715 non-null    object        
 4   LUGAR_DEL_HECHO      715 non-null    object        
 5   TIPO_DE_CALLE        715 non-null    object        
 6   COMUNA               715 non-null    int64         
 7   pos x                715 non-null    object        
 8   pos y                715 non-null    object        
 9   PARTICIPANTES        715 non-null    object        
 10  VICTIMA_x            715 non-null    object        
 11  ACUSADO              715 non-null    object        
 12  AÑO                  715 non-null    int32         
 13  ROL                  715 non-null    obj

#### En este punto podemos generar una columna con valores que indiquen los semestres, esto para satisfacer uno de los KPIs propuestos.

In [21]:
df_accidentes["SEMESTRE"] = [1 if month <= 6 else 2 for month in df_accidentes["FECHA"].dt.month]
df_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 715 entries, 0 to 716
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   715 non-null    object        
 1   N_VICTIMAS           715 non-null    int64         
 2   FECHA                715 non-null    datetime64[ns]
 3   HORA                 715 non-null    object        
 4   LUGAR_DEL_HECHO      715 non-null    object        
 5   TIPO_DE_CALLE        715 non-null    object        
 6   COMUNA               715 non-null    int64         
 7   pos x                715 non-null    object        
 8   pos y                715 non-null    object        
 9   PARTICIPANTES        715 non-null    object        
 10  VICTIMA_x            715 non-null    object        
 11  ACUSADO              715 non-null    object        
 12  AÑO                  715 non-null    int32         
 13  ROL                  715 non-null    obj

#### Ahora debemos organizar la estructura de las direcciones de la columna "LUGAR_DEL_HECHO".

In [22]:
# Reemplazamos las comas de la columna por espacios vacios, para después reorganizarlas
df_accidentes["LUGAR_DEL_HECHO"] = df_accidentes["LUGAR_DEL_HECHO"].str.replace(',', '')

In [23]:
# Definimos una función para reorganizar la estructura de los valores de la columna
def reorganizar_direcc(address):
    words = address.split()
    if "Av." in words:
        words.remove("Av.")
        words = ["Av."] + words
    return " ".join(words)

# Aplicamos la función a la columna y visualizamos
df_accidentes["LUGAR_DEL_HECHO"] = df_accidentes["LUGAR_DEL_HECHO"].apply(reorganizar_direcc)
df_accidentes.head(2)

,ID,N_VICTIMAS,FECHA,HORA,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA_x,ACUSADO,AÑO,ROL,SEXO,EDAD,FECHA_FALLECIMIENTO,SEMESTRE
0,2016-0001,1,2016-01-01,04:00:00,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,8,-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO,2016,CONDUCTOR,MASCULINO,menos de 20,2016-01-01,1
1,2016-0002,1,2016-01-02,01:15:00,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,9,-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS,2016,CONDUCTOR,MASCULINO,61-80,2016-01-02,1


#### Por último, podemos estandarizar los valores de la columna "HORA".

In [24]:
# Reemplazamos aquellos valores que tienen el formato incorrecto
formato_incorrecto = pd.to_datetime(df_accidentes["HORA"], errors="coerce").isnull()
df_accidentes.loc[~formato_incorrecto, "HORA"] = pd.to_datetime(df_accidentes.loc[~formato_incorrecto, "HORA"]).dt.strftime('%H:%M:%S')
df_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 715 entries, 0 to 716
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   715 non-null    object        
 1   N_VICTIMAS           715 non-null    int64         
 2   FECHA                715 non-null    datetime64[ns]
 3   HORA                 715 non-null    object        
 4   LUGAR_DEL_HECHO      715 non-null    object        
 5   TIPO_DE_CALLE        715 non-null    object        
 6   COMUNA               715 non-null    int64         
 7   pos x                715 non-null    object        
 8   pos y                715 non-null    object        
 9   PARTICIPANTES        715 non-null    object        
 10  VICTIMA_x            715 non-null    object        
 11  ACUSADO              715 non-null    object        
 12  AÑO                  715 non-null    int32         
 13  ROL                  715 non-null    obj

#### En este punto tenemos un dataframe limpio para hacer parte del análisis multivariado en el EDA y para satisfacer los KPIs propuestos. Lo disponibilizamos.

In [25]:
# df_accidentes.to_csv("Data/df_accidentes.csv")

##### El segundo paso de este ETL sería extraer y disponibilizar los datos con el histórico de la población y, por tanto, las tasasde crecimiento de dicha población.

##### Los datos de la población podemos obtenerlos de Wikiedia.

In [26]:
# URL de la página que contiene la lista de comunas de Buenos Aires por población
url = "https://es.wikipedia.org/wiki/Anexo:Comunas_de_la_ciudad_de_Buenos_Aires_por_poblaci%C3%B3n"

# Realizamos una solicitud GET a la URL y obtenemos el contenido de la página
request = requests.get(url)

# Creamos un objeto BeautifulSoup para analizar el contenido HTML de la página
sopa = BeautifulSoup(request.text, "html.parser")

# Utilizamos pandas para leer las tablas HTML de la URL
paginas = pd.read_html(url)
df_datos_comunas = paginas[0]
df_datos_comunas


Evolución demográfica e histórica de los habitantes de las Comunas de la Ciudad Autónoma de Buenos Aires (CABA)[1]​  \
                                                                                                         Puesto (2022)   
0                                                  1°                                                                    
1                                                  2°                                                                    
2                                                  3°                                                                    
3                                                  4°                                                                    
4                                                  5°                                                                    
5                                                  6°                                                                    
6                                                  7°                                                                    
7                                                  8°                                                                    
8                                                  9°                                                                    
9                                                 10°                                                                    
10                                                11°                                                                    
11                                                12°                                                                    
12                                                13°                                                                    
13                                                14°                                                                    
14                                                15°                                                                    
15                                              TOTAL                                                                    

                                                           \
       Comuna CENSO 1991 CENSO 2001 CENSO 2010 CENSO 2022   
0   Comuna 13    250 224    228 226    231 331    265 199   
1   Comuna 14    256 927    225 245    225 970    249 016   
2   Comuna 12    198 185    191 122    200 116    236 294   
3    Comuna 4    215 223    215 046    218 245    230 945   
4    Comuna 1    200 689    171 985    205 886    223 282   
5    Comuna 7    198 489    197 333    220 591    216 832   
6    Comuna 8    151 078    161 642    187 237    204 842   
7    Comuna 6    183 740    170 309    176 076    203 784   
8   Comuna 11    199 049    189 666    189 832    203 491   
9   Comuna 15    195 346    183 110    182 574    197 903   
10   Comuna 3    200 275    184 015    187 537    195 462   
11   Comuna 5    188 342    173 769    179 005    193 859   
12  Comuna 10    167 671    163 209    166 022    171 797   
13   Comuna 9    161 518    155 967    161 797    169 538   
14   Comuna 2    198 647    165 494    157 932    158 368   
15       CABA  2 965 403  2 776 138  2 890 151  3 120 612   

                                             
   Crecimiento 2010-2022 Unnamed: 7_level_1  
0              + 14,64 %                NaN  
1              + 10,19 %                NaN  
2              + 18,07 %                NaN  
3               + 5,81 %                NaN  
4               + 8,44 %                NaN  
5               - 1,70 %                NaN  
6               + 9,40 %                NaN  
7              + 15,73 %                NaN  
8               + 7,19 %                NaN  
9               + 8,39 %                NaN  
10              + 4,22 %                NaN  
11              + 8,29 %                NaN  
12              + 3,47 %                NaN  
13              +

In [27]:
# Disponibilizamos un archivo csv que volveremos a leer
# Esto con el objetivo de eliminar el índice jerárquico
df_datos_comunas.to_csv("Data/df_datos_comunas.csv", index=False)

In [28]:
df_datos_comunas = pd.read_csv("Data/df_datos_comunas.csv")
df_datos_comunas

,Evolución demográfica e histórica de los habitantes de las Comunas de la Ciudad Autónoma de Buenos Aires (CABA)[1]​,Evolución demográfica e histórica de los habitantes de las Comunas de la Ciudad Autónoma de Buenos Aires (CABA)[1]​.1,Evolución demográfica e histórica de los habitantes de las Comunas de la Ciudad Autónoma de Buenos Aires (CABA)[1]​.2,Evolución demográfica e histórica de los habitantes de las Comunas de la Ciudad Autónoma de Buenos Aires (CABA)[1]​.3,Evolución demográfica e histórica de los habitantes de las Comunas de la Ciudad Autónoma de Buenos Aires (CABA)[1]​.4,Evolución demográfica e histórica de los habitantes de las Comunas de la Ciudad Autónoma de Buenos Aires (CABA)[1]​.5,Evolución demográfica e histórica de los habitantes de las Comunas de la Ciudad Autónoma de Buenos Aires (CABA)[1]​.6,Evolución demográfica e histórica de los habitantes de las Comunas de la Ciudad Autónoma de Buenos Aires (CABA)[1]​.7
0,Puesto (2022),Comuna,CENSO 1991,CENSO 2001,CENSO 2010,CENSO 2022,Crecimiento 2010-2022,Unnamed: 7_level_1
1,1°,Comuna 13,250 224,228 226,231 331,265 199,"+ 14,64 %",NaN
2,2°,Comuna 14,256 927,225 245,225 970,249 016,"+ 10,19 %",NaN
3,3°,Comuna 12,198 185,191 122,200 116,236 294,"+ 18,07 %",NaN
4,4°,Comuna 4,215 223,215 046,218 245,230 945,"+ 5,81 %",NaN
5,5°,Comuna 1,200 689,171 985,205 886,223 282,"+ 8,44 %",NaN
6,6°,Comuna 7,198 489,197 333,220 591,216 832,"- 1,70 %",NaN
7,7°,Comuna 8,151 078,161 642,187 237,204 842,"+ 9,40 %",NaN
8,8°,Comuna 6,183 740,170 309,176 076,203 784,"+ 15,73 %",NaN
9,9°,Comuna 11,199 049,189 666,189 832,203 491,"+ 7,19 %",NaN


##### Una vez que tenemos el dataframe, podemos eliminar y renombrar columnas innecesarias.

In [29]:
# Renombramos las columnasDataFrame "datos_comunas"
df_datos_comunas = df_datos_comunas.rename(columns={
    "Evolución demográfica e histórica de los habitantes de las Comunas de la Ciudad Autónoma de Buenos Aires (CABA)[1]\u200b": "puesto (2022)",
    "Evolución demográfica e histórica de los habitantes de las Comunas de la Ciudad Autónoma de Buenos Aires (CABA)[1]\u200b.1": "Comuna",
    "Evolución demográfica e histórica de los habitantes de las Comunas de la Ciudad Autónoma de Buenos Aires (CABA)[1]\u200b.2": "CENSO 1991",
    "Evolución demográfica e histórica de los habitantes de las Comunas de la Ciudad Autónoma de Buenos Aires (CABA)[1]\u200b.3": "CENSO 2001",
    "Evolución demográfica e histórica de los habitantes de las Comunas de la Ciudad Autónoma de Buenos Aires (CABA)[1]\u200b.4": "CENSO 2010",
    "Evolución demográfica e histórica de los habitantes de las Comunas de la Ciudad Autónoma de Buenos Aires (CABA)[1]\u200b.5": "CENSO 2022",
    "Evolución demográfica e histórica de los habitantes de las Comunas de la Ciudad Autónoma de Buenos Aires (CABA)[1]\u200b.6": "CRECIMIENTO 2010-2022"
})

df_datos_comunas = df_datos_comunas.drop(columns=[
    "Evolución demográfica e histórica de los habitantes de las Comunas de la Ciudad Autónoma de Buenos Aires (CABA)[1]\u200b.7",
    "CENSO 2001",
    "CENSO 1991"
])

# Eliminamos el primer registro de la columna y reseteamos el índice
df_datos_comunas = df_datos_comunas.drop(0).reset_index(drop=True)

df_datos_comunas.head()

,puesto (2022),Comuna,CENSO 2010,CENSO 2022,CRECIMIENTO 2010-2022
0,1°,Comuna 13,231 331,265 199,"+ 14,64 %"
1,2°,Comuna 14,225 970,249 016,"+ 10,19 %"
2,3°,Comuna 12,200 116,236 294,"+ 18,07 %"
3,4°,Comuna 4,218 245,230 945,"+ 5,81 %"
4,5°,Comuna 1,205 886,223 282,"+ 8,44 %"


##### Necesitamos hacer un cálculo para obtener el censo del 2021.

In [30]:
# Convertimos la columna 'CENSO 2022' a float
df_datos_comunas["CENSO 2022"] = df_datos_comunas["CENSO 2022"].str.replace("\xa0", "").astype(float)

# Poblacion total de 2022
censo_2022 = 3120612

# Tasa de crecimiento total entre 2010 y 2022
tasa_crecimiento_total = 7.97 / 100

# Calcular la tasa de crecimiento anual
tasa_crecimiento_anual = (1 + tasa_crecimiento_total) ** (1 / 12) - 1

# Crear una nueva columna con los valores estimados para 2021
df_datos_comunas["CENSO 2021"] = df_datos_comunas["CENSO 2022"] / (1 + tasa_crecimiento_anual)

# Cambiamos el tipo de formato de la columna
df_datos_comunas["CENSO 2021"] = df_datos_comunas["CENSO 2021"].astype(int)

df_datos_comunas.head()

,puesto (2022),Comuna,CENSO 2010,CENSO 2022,CRECIMIENTO 2010-2022,CENSO 2021
0,1°,Comuna 13,231 331,265199.0,"+ 14,64 %",263509
1,2°,Comuna 14,225 970,249016.0,"+ 10,19 %",247429
2,3°,Comuna 12,200 116,236294.0,"+ 18,07 %",234788
3,4°,Comuna 4,218 245,230945.0,"+ 5,81 %",229473
4,5°,Comuna 1,205 886,223282.0,"+ 8,44 %",221859


##### Una vez que se tiene el censo del 2021 se necesita calcular el factor de crecimiento por comuna.

In [31]:
data = {
    "Comuna": ["Comuna 13", "Comuna 14", "Comuna 12", "Comuna 4", "Comuna 1", "Comuna 7", "Comuna 8", "Comuna 6", "Comuna 11", "Comuna 15", "Comuna 3", "Comuna 5", "Comuna 10", "Comuna 9", "Comuna 2"],
    "Poblacion 2010": [231331, 225970, 200116, 218245, 205886, 220591, 187237, 176076, 189832, 182574, 187537, 179005, 166022, 161797, 157932],
    "Crecimiento 2010-2022": [14.64, 10.19, 18.07, 5.81, 8.44, -1.70, 9.40, 15.73, 7.19, 8.39, 4.22, 8.29, 3.47, 4.78, 0.27]
}

df_datos_comunas = pd.DataFrame(data)

# Convertir tasas de crecimiento en factores de crecimiento
df_datos_comunas["Factor de crecimiento"] = 1 + df_datos_comunas["Crecimiento 2010-2022"] / 100

# Calcular población para cada año
for año in range(2016, 2022):
    n = año - 2010
    df_datos_comunas[f"Poblacion {año}"] = df_datos_comunas["Poblacion 2010"] * df_datos_comunas["Factor de crecimiento"] ** (n / 12)

df_datos_comunas

,Comuna,Poblacion 2010,Crecimiento 2010-2022,Factor de crecimiento,Poblacion 2016,Poblacion 2017,Poblacion 2018,Poblacion 2019,Poblacion 2020,Poblacion 2021
0,Comuna 13,231331,14.64,1.1464,247686.264822,250522.424102,253391.059141,256292.541807,259227.248224,262195.558826
1,Comuna 14,225970,10.19,1.1019,237203.928356,239129.813827,241071.335779,243028.621166,245001.797975,246990.995229
2,Comuna 12,200116,18.07,1.1807,217446.086117,220476.966723,223550.093374,226666.054919,229825.448413,233028.879235
3,Comuna 4,218245,5.81,1.0581,224495.510321,225554.529496,226618.544413,227687.578638,228761.655849,229840.799836
4,Comuna 1,205886,8.44,1.0844,214398.415091,215850.982427,217313.391029,218785.707572,220267.999184,221760.333447
5,Comuna 7,220591,-1.70,0.9830,218707.939187,218395.662249,218083.831189,217772.445369,217461.504154,217151.006909
6,Comuna 8,187237,9.40,1.0940,195839.519558,197311.218479,198793.976954,200287.878093,201793.005632,203309.443935
7,Comuna 6,176076,15.73,1.1573,189418.825818,191738.931523,194087.455157,196464.744796,198871.152783,201307.035776
8,Comuna 11,189832,7.19,1.0719,196538.011991,197678.489874,198825.585764,199979.338063,201139.785398,202306.966618
9,Comuna 15,182574,8.39,1.0839,190078.737500,191359.182404,192648.252886,193946.007054,195252.503402,196567.800821


##### Cambiamos los tipos de datos de las columnas.

In [32]:
# Columnas de años a modificar
columnas_a_modificar = ["Poblacion 2016", "Poblacion 2017", "Poblacion 2018", "Poblacion 2019", "Poblacion 2020", "Poblacion 2021"]

# Cambiar el tipo de las columnas a int64
for columna in columnas_a_modificar:
    df_datos_comunas[columna] = df_datos_comunas[columna].astype("int64")

In [33]:
df_datos_comunas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Comuna                 15 non-null     object 
 1   Poblacion 2010         15 non-null     int64  
 2   Crecimiento 2010-2022  15 non-null     float64
 3   Factor de crecimiento  15 non-null     float64
 4   Poblacion 2016         15 non-null     int64  
 5   Poblacion 2017         15 non-null     int64  
 6   Poblacion 2018         15 non-null     int64  
 7   Poblacion 2019         15 non-null     int64  
 8   Poblacion 2020         15 non-null     int64  
 9   Poblacion 2021         15 non-null     int64  
dtypes: float64(2), int64(7), object(1)
memory usage: 1.3+ KB


##### Eliminamos la columna innecesaria y modificamos los valores de la columna "Comuna" para cambiar su tipo de dato.

In [34]:
df_datos_comunas = df_datos_comunas.drop(columns=["Factor de crecimiento"])
df_datos_comunas["Comuna"] = df_datos_comunas["Comuna"].str.replace("Comuna", "")
df_datos_comunas["Comuna"] = df_datos_comunas["Comuna"].astype("int64")
df_datos_comunas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Comuna                 15 non-null     int64  
 1   Poblacion 2010         15 non-null     int64  
 2   Crecimiento 2010-2022  15 non-null     float64
 3   Poblacion 2016         15 non-null     int64  
 4   Poblacion 2017         15 non-null     int64  
 5   Poblacion 2018         15 non-null     int64  
 6   Poblacion 2019         15 non-null     int64  
 7   Poblacion 2020         15 non-null     int64  
 8   Poblacion 2021         15 non-null     int64  
dtypes: float64(1), int64(8)
memory usage: 1.2 KB


##### Ya en este punto podemos disponibilizar el dataframe en un archivo CSV para ser consultado de ser neceario.

In [35]:
df_datos_comunas.to_csv("Data/poblaciónCABA")

##### A partir de este punto podemos empezar a construir la data para uno de nuestros KPIs

In [36]:
df_accidentes_suma = df_accidentes.groupby(["AÑO", "SEMESTRE"])["N_VICTIMAS"].sum().reset_index()
df_accidentes_suma

,AÑO,SEMESTRE,N_VICTIMAS
0,2016,1,67
1,2016,2,82
2,2017,1,81
3,2017,2,79
4,2018,1,76
5,2018,2,85
6,2019,1,59
7,2019,2,46
8,2020,1,31
9,2020,2,56


In [37]:
# Eliminar columnas del DataFrame directamente
df_datos_comunas = df_datos_comunas.drop(columns=["Poblacion 2010", "Crecimiento 2010-2022"])
df_datos_comunas

,Comuna,Poblacion 2016,Poblacion 2017,Poblacion 2018,Poblacion 2019,Poblacion 2020,Poblacion 2021
0,13,247686,250522,253391,256292,259227,262195
1,14,237203,239129,241071,243028,245001,246990
2,12,217446,220476,223550,226666,229825,233028
3,4,224495,225554,226618,227687,228761,229840
4,1,214398,215850,217313,218785,220267,221760
5,7,218707,218395,218083,217772,217461,217151
6,8,195839,197311,198793,200287,201793,203309
7,6,189418,191738,194087,196464,198871,201307
8,11,196538,197678,198825,199979,201139,202306
9,15,190078,191359,192648,193946,195252,196567


In [38]:
# Generamos un dataframe con la población anualizada
comunas_suma = df_datos_comunas[["Poblacion 2016", "Poblacion 2017", "Poblacion 2018", "Poblacion 2019", "Poblacion 2020", "Poblacion 2021"]].sum()
df_poblacion_suma = comunas_suma.to_frame().reset_index()

# Definimos las columnas de nuestro dataframe
df_poblacion_suma.columns = ["AÑO", "POBLACION TOTAL"]

# Eliminamos la palabra "población" de los valores
df_poblacion_suma["AÑO"] = df_poblacion_suma["AÑO"].str.replace("Poblacion", "", case=False)
df_poblacion_suma

,AÑO,POBLACION TOTAL
0,2016,3002178
1,2017,3021444
2,2018,3040889
3,2019,3060508
4,2020,3080306
5,2021,3100282


##### En este punto podemos hacer un merged entre el dataframe con los semestres y el dataframe con la boblacion anualizada.

In [39]:
df_accidentes_suma["AÑO"] = df_accidentes_suma["AÑO"].astype("int64")
df_accidentes_suma.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   AÑO         12 non-null     int64
 1   SEMESTRE    12 non-null     int64
 2   N_VICTIMAS  12 non-null     int64
dtypes: int64(3)
memory usage: 420.0 bytes


In [40]:
# Antes convertimos el tipo de dato de la columna "AÑO" de cada dataframe, a int
df_accidentes_suma["AÑO"] = df_accidentes_suma["AÑO"].astype("int64")
df_poblacion_suma["AÑO"] = df_poblacion_suma["AÑO"].astype("int64")

# Hacemos el merged
df_kpi_1 = df_accidentes_suma.merge(df_poblacion_suma, on="AÑO")
df_kpi_1

,AÑO,SEMESTRE,N_VICTIMAS,POBLACION TOTAL
0,2016,1,67,3002178
1,2016,2,82,3002178
2,2017,1,81,3021444
3,2017,2,79,3021444
4,2018,1,76,3040889
5,2018,2,85,3040889
6,2019,1,59,3060508
7,2019,2,46,3060508
8,2020,1,31,3080306
9,2020,2,56,3080306


##### Creamos una nueva columna con la tasa de accidentes.

In [41]:
df_kpi_1["TASA"] = (df_kpi_1["N_VICTIMAS"] / df_kpi_1["POBLACION TOTAL"]) * 100000
df_kpi_1

,AÑO,SEMESTRE,N_VICTIMAS,POBLACION TOTAL,TASA
0,2016,1,67,3002178,2.231713
1,2016,2,82,3002178,2.731350
2,2017,1,81,3021444,2.680837
3,2017,2,79,3021444,2.614644
4,2018,1,76,3040889,2.499269
5,2018,2,85,3040889,2.795235
6,2019,1,59,3060508,1.927785
7,2019,2,46,3060508,1.503018
8,2020,1,31,3080306,1.006394
9,2020,2,56,3080306,1.818001


##### Ahora agregamos una columna con la tasa del periodo anterior.

In [42]:
df_kpi_1["TASA ANTERIOR"] = df_kpi_1["TASA"].shift(periods=1, fill_value=0)
df_kpi_1

,AÑO,SEMESTRE,N_VICTIMAS,POBLACION TOTAL,TASA,TASA ANTERIOR
0,2016,1,67,3002178,2.231713,0.000000
1,2016,2,82,3002178,2.731350,2.231713
2,2017,1,81,3021444,2.680837,2.731350
3,2017,2,79,3021444,2.614644,2.680837
4,2018,1,76,3040889,2.499269,2.614644
5,2018,2,85,3040889,2.795235,2.499269
6,2019,1,59,3060508,1.927785,2.795235
7,2019,2,46,3060508,1.503018,1.927785
8,2020,1,31,3080306,1.006394,1.503018
9,2020,2,56,3080306,1.818001,1.006394


##### Ahora dedemos agregar una columna con la variación de la tasa.

In [43]:
# Creamos la columna haciendo el cálulo entre las columnas "TASA" y "TASA ANTERIOR"
df_kpi_1['VARIACION_TASA'] = (df_kpi_1["TASA"] - df_kpi_1["TASA ANTERIOR"]) / df_kpi_1["TASA ANTERIOR"] * 100
df_kpi_1

,AÑO,SEMESTRE,N_VICTIMAS,POBLACION TOTAL,TASA,TASA ANTERIOR,VARIACION_TASA
0,2016,1,67,3002178,2.231713,0.000000,inf
1,2016,2,82,3002178,2.731350,2.231713,22.388060
2,2017,1,81,3021444,2.680837,2.731350,-1.849378
3,2017,2,79,3021444,2.614644,2.680837,-2.469136
4,2018,1,76,3040889,2.499269,2.614644,-4.412637
5,2018,2,85,3040889,2.795235,2.499269,11.842105
6,2019,1,59,3060508,1.927785,2.795235,-31.033191
7,2019,2,46,3060508,1.503018,1.927785,-22.033898
8,2020,1,31,3080306,1.006394,1.503018,-33.041839
9,2020,2,56,3080306,1.818001,1.006394,80.645161


In [44]:
df_kpi_1["VARIACION_TASA"] = df_kpi_1["VARIACION_TASA"].replace([np.inf, -np.inf], 0)
df_kpi_1["VARIACION_TASA"] = round(df_kpi_1["VARIACION_TASA"], 2)
df_kpi_1

,AÑO,SEMESTRE,N_VICTIMAS,POBLACION TOTAL,TASA,TASA ANTERIOR,VARIACION_TASA
0,2016,1,67,3002178,2.231713,0.000000,0.00
1,2016,2,82,3002178,2.731350,2.231713,22.39
2,2017,1,81,3021444,2.680837,2.731350,-1.85
3,2017,2,79,3021444,2.614644,2.680837,-2.47
4,2018,1,76,3040889,2.499269,2.614644,-4.41
5,2018,2,85,3040889,2.795235,2.499269,11.84
6,2019,1,59,3060508,1.927785,2.795235,-31.03
7,2019,2,46,3060508,1.503018,1.927785,-22.03
8,2020,1,31,3080306,1.006394,1.503018,-33.04
9,2020,2,56,3080306,1.818001,1.006394,80.65


##### En este punto podemos disponibilizar un archivo CSV con el dataframe del primer KPI, que graficaremos en el dashboard con la herramienta de BI.

In [46]:
df_kpi_1.to_csv("Data/df_kpi_1.csv")

##### Empezamos a trabajar en el dataframe para suplir el segundo KPI propuesto.

In [49]:
df_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 715 entries, 0 to 716
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   715 non-null    object        
 1   N_VICTIMAS           715 non-null    int64         
 2   FECHA                715 non-null    datetime64[ns]
 3   HORA                 715 non-null    object        
 4   LUGAR_DEL_HECHO      715 non-null    object        
 5   TIPO_DE_CALLE        715 non-null    object        
 6   COMUNA               715 non-null    int64         
 7   pos x                715 non-null    object        
 8   pos y                715 non-null    object        
 9   PARTICIPANTES        715 non-null    object        
 10  VICTIMA_x            715 non-null    object        
 11  ACUSADO              715 non-null    object        
 12  AÑO                  715 non-null    int32         
 13  ROL                  715 non-null    obj

In [53]:
# Seleccionamos aquellos registros del dataframe en donde la víctima iba en moto
df_kpi_2 = df_accidentes[df_accidentes["VICTIMA_x"] == "MOTO"]
df_kpi_2.head(2)

,ID,N_VICTIMAS,FECHA,HORA,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA_x,ACUSADO,AÑO,ROL,SEXO,EDAD,FECHA_FALLECIMIENTO,SEMESTRE
0,2016-0001,1,2016-01-01,04:00:00,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,8,-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO,2016,CONDUCTOR,MASCULINO,menos de 20,2016-01-01,1
2,2016-0003,1,2016-01-03,07:00:00,AV ENTRE RIOS 2034,AVENIDA,1,-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO,2016,CONDUCTOR,MASCULINO,21-40,2016-01-03,1


In [54]:
# Agrupamos las víctimas en los años correspondientes
df_kpi_2 = df_kpi_2.groupby(["AÑO"])["N_VICTIMAS"].sum().reset_index()
df_kpi_2

,AÑO,N_VICTIMAS
0,2016,67
1,2017,62
2,2018,61
3,2019,49
4,2020,30
5,2021,46


In [55]:
# Creamos una columna con el número de víctimas del periodo anterior
df_kpi_2["VICTIMAS_ANTERIORES"] = df_kpi_2["N_VICTIMAS"].shift(periods=1, fill_value=0)
df_kpi_2

,AÑO,N_VICTIMAS,VICTIMAS_ANTERIORES
0,2016,67,0
1,2017,62,67
2,2018,61,62
3,2019,49,61
4,2020,30,49
5,2021,46,30


In [56]:
# Creamos una columna con la variación de las víctimas
df_kpi_2["VARIACION_DE_VICTIMAS"] = (df_kpi_2["N_VICTIMAS"] - df_kpi_2["VICTIMAS_ANTERIORES"]) / df_kpi_2["VICTIMAS_ANTERIORES"] * 100
df_kpi_2

,AÑO,N_VICTIMAS,VICTIMAS_ANTERIORES,VARIACION_DE_VICTIMAS
0,2016,67,0,inf
1,2017,62,67,-7.462687
2,2018,61,62,-1.612903
3,2019,49,61,-19.672131
4,2020,30,49,-38.775510
5,2021,46,30,53.333333


In [58]:
# Reemplazamos los valores infinitos y redondeamos la columna a dos decimales
df_kpi_2["VARIACION_DE_VICTIMAS"] = df_kpi_2["VARIACION_DE_VICTIMAS"].replace([np.inf, -np.inf], 0)
df_kpi_2["VARIACION_DE_VICTIMAS"] = round(df_kpi_2["VARIACION_DE_VICTIMAS"], 2)
df_kpi_2

,AÑO,N_VICTIMAS,VICTIMAS_ANTERIORES,VARIACION_DE_VICTIMAS
0,2016,67,0,0.00
1,2017,62,67,-7.46
2,2018,61,62,-1.61
3,2019,49,61,-19.67
4,2020,30,49,-38.78
5,2021,46,30,53.33


##### Aquí podemos disponibilizar un archivo CSV con el dataframe del segundo KPI.

In [59]:
df_kpi_2.to_csv("Data/df_kpi_2.csv")